## Agrupamiento_Jerárquico

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score, precision_score, recall_score
from itertools import combinations
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler  
import time

# Cargar datos
df_datos = pd.read_csv("datos/credit_risk_dataset.csv")

# Definir columnas importantes
columnas_importantes = ['person_age', 'person_income', 'loan_status', 'cb_person_cred_hist_length', 'loan_grade']

# Preprocesamiento de datos
imputer = SimpleImputer(strategy='mean')
df_datos[columnas_importantes] = imputer.fit_transform(df_datos[columnas_importantes])

# Normalizar los datos
arr_normalizado = MinMaxScaler().fit_transform(df_datos[columnas_importantes])

# Crear DataFrame con los datos normalizados
df_normalizado = pd.DataFrame(arr_normalizado, columns=columnas_importantes)

# Función para entrenar y evaluar modelo de Agrupamiento Jerárquico
def train_evaluate_agglomerative(X_train, X_test, y_train, y_test):
    start_model_generation = time.time()
    modelo_aglomerativo = AgglomerativeClustering(n_clusters=2, linkage='complete', metric='euclidean').fit(X_train)
    end_model_generation = time.time()
    model_generation_time = end_model_generation - start_model_generation
    
    start_test_prediction = time.time()
    labels_train = modelo_aglomerativo.labels_
    labels_test = modelo_aglomerativo.fit_predict(X_test)
    
    # Mapeo de clusters a valores objetivo
    cluster_0_label = np.argmax(np.bincount(y_train[labels_train == 0]))
    cluster_1_label = np.argmax(np.bincount(y_train[labels_train == 1]))
    y_pred_test_mapped = [cluster_0_label if label == 0 else cluster_1_label for label in labels_test]
    
    # Calcular métricas de validación
    accuracy = accuracy_score(y_test, y_pred_test_mapped)
    precision = precision_score(y_test, y_pred_test_mapped, average='weighted', zero_division=1)
    recall = recall_score(y_test, y_pred_test_mapped, average='weighted')
    end_test_prediction = time.time()
    test_prediction_time = end_test_prediction - start_test_prediction
    
    return accuracy, precision, recall, model_generation_time, test_prediction_time

# DataFrame para almacenar resultados
resultados = []

# Bucle para generar todas las combinaciones posibles de columnas
for r in range(1, len(columnas_importantes) + 1):
    combinaciones_columnas = combinations(columnas_importantes, r)
    # Generar combinaciones de longitud r
    for cols in combinaciones_columnas:
        cols = list(cols)
        X_train_subset = df_normalizado.loc[:, cols]  # Seleccionar las columnas de entrenamiento
        X_test_subset = df_normalizado.loc[:, cols]   # Seleccionar las columnas de prueba
        y_train_subset = df_datos['loan_status']  # Variable objetivo para entrenamiento
        y_test_subset = df_datos['loan_status']   # Variable objetivo para prueba
        
        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X_train_subset, y_train_subset, test_size=0.2, random_state=100)
        
        accuracy, precision, recall, model_generation_time, test_prediction_time = train_evaluate_agglomerative(X_train, X_test, y_train, y_test)
        resultados.append([cols, accuracy, precision, recall, model_generation_time, test_prediction_time])

# Crear DataFrame final
df_resultados = pd.DataFrame(resultados, columns=['Columnas', 'Exactitud', 'Precisión', 'Sensibilidad', 'Tiempo_Modelo', 'Tiempo_Prueba'])

# Ordenar DataFrame por Exactitud
df_resultados = df_resultados.sort_values(by='Exactitud', ascending=False)

# Mostrar DataFrame final
for idx, row in df_resultados.iterrows():
    print("Combinación de Columnas: {}".format(row['Columnas']))
    print("Exactitud: {:.2f}".format(row['Exactitud']))
    print("Precisión: {:.2f}".format(row['Precisión']))
    print("Sensibilidad: {:.2f}".format(row['Sensibilidad']))
    print("Tiempo de Generación del Modelo: {:.6f} segundos".format(row['Tiempo_Modelo']))
    print("Tiempo de Prueba: {:.6f} segundos".format(row['Tiempo_Prueba']))
    print("---------------------------------------")

Combinación de Columnas: ['loan_status']
Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Tiempo de Generación del Modelo: 5.520315 segundos
Tiempo de Prueba: 0.444909 segundos
---------------------------------------
Combinación de Columnas: ['person_age', 'loan_status', 'cb_person_cred_hist_length', 'loan_grade']
Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Tiempo de Generación del Modelo: 12.549717 segundos
Tiempo de Prueba: 0.740368 segundos
---------------------------------------
Combinación de Columnas: ['person_age', 'person_income', 'loan_status', 'loan_grade']
Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Tiempo de Generación del Modelo: 41.740444 segundos
Tiempo de Prueba: 1.224626 segundos
---------------------------------------
Combinación de Columnas: ['person_age', 'loan_status']
Exactitud: 1.00
Precisión: 1.00
Sensibilidad: 1.00
Tiempo de Generación del Modelo: 6.708407 segundos
Tiempo de Prueba: 0.591138 segundos
---------------------------------------
Combi